In [1]:
spark

In [4]:
firstDataFrame = sqlContext.range(1000000)

print(firstDataFrame)

DataFrame[id: bigint]


In [5]:
secondDataFrame = firstDataFrame.selectExpr("(id * 2) as value")

In [6]:
print(firstDataFrame.take(5))
print(secondDataFrame.take(5))

[Row(id=0), Row(id=1), Row(id=2), Row(id=3), Row(id=4)]
[Row(value=0), Row(value=2), Row(value=4), Row(value=6), Row(value=8)]


In [11]:
dataPath = "./diamonds.csv"
diamonds = spark.read.format("csv").option("header","true").option("inferSchema", "true").load(dataPath)

display(diamonds)

DataFrame[carat: double, cut: string, color: string, clarity: string, depth: double, table: double, price: int, x: double, y: double, z: double]

In [12]:
df1 = diamonds.groupBy("cut", "color").avg("price") # a simple grouping

df2 = df1.join(diamonds, on='color', how='inner').select("`avg(price)`", "carat")

In [14]:
df2.explain()

== Physical Plan ==
*(3) Project [avg(price)#81, carat#85]
+- *(3) BroadcastHashJoin [color#52], [color#87], Inner, BuildRight
   :- *(3) HashAggregate(keys=[cut#51, color#52], functions=[avg(cast(price#56 as bigint))])
   :  +- Exchange hashpartitioning(cut#51, color#52, 200)
   :     +- *(1) HashAggregate(keys=[cut#51, color#52], functions=[partial_avg(cast(price#56 as bigint))])
   :        +- *(1) Project [cut#51, color#52, price#56]
   :           +- *(1) Filter isnotnull(color#52)
   :              +- *(1) FileScan csv [cut#51,color#52,price#56] Batched: false, Format: CSV, Location: InMemoryFileIndex[file:/home/churtado/notebooks/pyspark/Databricks tutorial/diamonds.csv], PartitionFilters: [], PushedFilters: [IsNotNull(color)], ReadSchema: struct<cut:string,color:string,price:int>
   +- BroadcastExchange HashedRelationBroadcastMode(List(input[1, string, true]))
      +- *(2) Project [carat#85, color#87]
         +- *(2) Filter isnotnull(color#87)
            +- *(2) FileScan csv

In [15]:
df2.count()

269700

In [16]:
df2.cache()

DataFrame[avg(price): double, carat: double]

In [17]:
df2.count()

269700